# NFL Big Data Bowl 2026 - Geometric Neural Network Training

**The Breakthrough Architecture**

This notebook implements the Geometric Neural Network approach that achieved **0.559 Public LB**.

**Key Insight**: Football players follow geometric rules with learned corrections:
- Receivers -> Ball landing (geometric)
- Defenders -> Mirror receivers (geometric coupling)
- Others -> Momentum (physics)
- Model learns only CORRECTIONS to geometric baseline

**Architecture**:
- Proven GRU + Attention (0.59 base)
- 154 proven features (unchanged)
- +13 geometric features (the breakthrough)
- Train on corrections to geometric baseline

**Contents**:
1. Setup and Configuration
2. Geometric Baseline Computation
3. 167-Feature Engineering Pipeline
4. GNN-lite Neighbor Embeddings
5. Model Architecture
6. Training with 5-Fold CV
7. Evaluation

In [1]:
# Core imports
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import warnings
import os
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.cluster import KMeans

warnings.filterwarnings('ignore')

print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')

PyTorch: 2.7.0+cu128
CUDA available: True


## 1. Configuration

In [2]:
class Config:
    """Training configuration for Geometric Neural Network"""
    
    # Data paths
    DATA_DIR = Path('/mnt/raid0/Kaggle Big Data Bowl/data/raw')
    OUTPUT_DIR = Path('../models/gnn_geometric')
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    # Training
    SEED = 42
    N_FOLDS = 5
    BATCH_SIZE = 256
    EPOCHS = 200
    PATIENCE = 30
    LEARNING_RATE = 1e-3
    
    # Model
    WINDOW_SIZE = 10
    HIDDEN_DIM = 128
    MAX_FUTURE_HORIZON = 94
    
    # Field dimensions
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    
    # GNN parameters
    K_NEIGH = 6      # Number of neighbors
    RADIUS = 30.0    # Neighbor radius
    TAU = 8.0        # Distance weighting
    N_ROUTE_CLUSTERS = 7  # Route pattern clusters
    
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

cfg = Config()
set_seed(cfg.SEED)

print(f'Device: {cfg.DEVICE}')
print(f'Window size: {cfg.WINDOW_SIZE}')
print(f'K neighbors: {cfg.K_NEIGH}')
print(f'Route clusters: {cfg.N_ROUTE_CLUSTERS}')

Device: cuda
Window size: 10
K neighbors: 6
Route clusters: 7


## 2. Geometric Baseline - THE BREAKTHROUGH

The key insight: compute where players SHOULD end up based on geometry, then train the model to learn corrections.

In [3]:
def compute_geometric_endpoint(df):
    """
    Compute where each player SHOULD end up based on geometry.
    This is the deterministic part - no learning needed.
    
    Rules:
    - Targeted Receivers -> Ball landing spot
    - Defensive Coverage -> Mirror receiver position
    - Others -> Momentum-based projection
    """
    df = df.copy()
    
    # Time to play end
    if 'num_frames_output' in df.columns:
        t_total = df['num_frames_output'] / 10.0
    else:
        t_total = 3.0
    
    df['time_to_endpoint'] = t_total
    
    # Initialize with momentum (default rule)
    df['geo_endpoint_x'] = df['x'] + df['velocity_x'] * t_total
    df['geo_endpoint_y'] = df['y'] + df['velocity_y'] * t_total
    
    # Rule 1: Targeted Receivers converge to ball
    if 'ball_land_x' in df.columns:
        receiver_mask = df['player_role'] == 'Targeted Receiver'
        df.loc[receiver_mask, 'geo_endpoint_x'] = df.loc[receiver_mask, 'ball_land_x']
        df.loc[receiver_mask, 'geo_endpoint_y'] = df.loc[receiver_mask, 'ball_land_y']
        
        # Rule 2: Defenders mirror receivers (maintain offset)
        defender_mask = df['player_role'] == 'Defensive Coverage'
        has_mirror = df.get('mirror_offset_x', 0).notna() & (df.get('mirror_wr_dist', 50) < 15)
        coverage_mask = defender_mask & has_mirror
        
        df.loc[coverage_mask, 'geo_endpoint_x'] = (
            df.loc[coverage_mask, 'ball_land_x'] + 
            df.loc[coverage_mask, 'mirror_offset_x'].fillna(0)
        )
        df.loc[coverage_mask, 'geo_endpoint_y'] = (
            df.loc[coverage_mask, 'ball_land_y'] + 
            df.loc[coverage_mask, 'mirror_offset_y'].fillna(0)
        )
    
    # Clip to field boundaries
    df['geo_endpoint_x'] = df['geo_endpoint_x'].clip(cfg.FIELD_X_MIN, cfg.FIELD_X_MAX)
    df['geo_endpoint_y'] = df['geo_endpoint_y'].clip(cfg.FIELD_Y_MIN, cfg.FIELD_Y_MAX)
    
    return df

def add_geometric_features(df):
    """Add features that describe the geometric solution"""
    df = compute_geometric_endpoint(df)
    
    # Vector to geometric endpoint
    df['geo_vector_x'] = df['geo_endpoint_x'] - df['x']
    df['geo_vector_y'] = df['geo_endpoint_y'] - df['y']
    df['geo_distance'] = np.sqrt(df['geo_vector_x']**2 + df['geo_vector_y']**2)
    
    # Required velocity to reach geometric endpoint
    t = df['time_to_endpoint'] + 0.1
    df['geo_required_vx'] = df['geo_vector_x'] / t
    df['geo_required_vy'] = df['geo_vector_y'] / t
    
    # Current velocity vs required
    df['geo_velocity_error_x'] = df['geo_required_vx'] - df['velocity_x']
    df['geo_velocity_error_y'] = df['geo_required_vy'] - df['velocity_y']
    df['geo_velocity_error'] = np.sqrt(
        df['geo_velocity_error_x']**2 + df['geo_velocity_error_y']**2
    )
    
    # Required acceleration
    t_sq = t * t
    df['geo_required_ax'] = (2 * df['geo_vector_x'] / t_sq).clip(-10, 10)
    df['geo_required_ay'] = (2 * df['geo_vector_y'] / t_sq).clip(-10, 10)
    
    # Alignment with geometric path
    velocity_mag = np.sqrt(df['velocity_x']**2 + df['velocity_y']**2)
    geo_unit_x = df['geo_vector_x'] / (df['geo_distance'] + 0.1)
    geo_unit_y = df['geo_vector_y'] / (df['geo_distance'] + 0.1)
    df['geo_alignment'] = (
        df['velocity_x'] * geo_unit_x + df['velocity_y'] * geo_unit_y
    ) / (velocity_mag + 0.1)
    
    return df

print('Geometric baseline functions defined')

Geometric baseline functions defined


## 3. GNN-lite Neighbor Embeddings

Compute spatial relationships between players using distance-weighted aggregation.

In [4]:
def compute_neighbor_embeddings(input_df, k_neigh=6, radius=30.0, tau=8.0):
    """GNN-lite: Distance-weighted neighbor feature aggregation"""
    print('Computing GNN neighbor embeddings...')
    
    cols_needed = ['game_id', 'play_id', 'nfl_id', 'frame_id', 'x', 'y', 
                   'velocity_x', 'velocity_y', 'player_side']
    src = input_df[cols_needed].copy()
    
    # Get last frame for each player
    last = (src.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
               .groupby(['game_id', 'play_id', 'nfl_id'], as_index=False)
               .tail(1)
               .rename(columns={'frame_id': 'last_frame_id'})
               .reset_index(drop=True))
    
    # Self-join to get all player pairs
    tmp = last.merge(
        src.rename(columns={
            'frame_id': 'nb_frame_id', 'nfl_id': 'nfl_id_nb',
            'x': 'x_nb', 'y': 'y_nb', 
            'velocity_x': 'vx_nb', 'velocity_y': 'vy_nb', 
            'player_side': 'player_side_nb'
        }),
        left_on=['game_id', 'play_id', 'last_frame_id'],
        right_on=['game_id', 'play_id', 'nb_frame_id'],
        how='left'
    )
    
    # Remove self-connections
    tmp = tmp[tmp['nfl_id_nb'] != tmp['nfl_id']]
    
    # Compute relative features
    tmp['dx'] = tmp['x_nb'] - tmp['x']
    tmp['dy'] = tmp['y_nb'] - tmp['y']
    tmp['dvx'] = tmp['vx_nb'] - tmp['velocity_x']
    tmp['dvy'] = tmp['vy_nb'] - tmp['velocity_y']
    tmp['dist'] = np.sqrt(tmp['dx']**2 + tmp['dy']**2)
    
    # Filter by radius
    tmp = tmp[np.isfinite(tmp['dist']) & (tmp['dist'] > 1e-6)]
    if radius is not None:
        tmp = tmp[tmp['dist'] <= radius]
    
    # Ally/opponent flag
    tmp['is_ally'] = (tmp['player_side_nb'] == tmp['player_side']).astype(np.float32)
    
    # Rank neighbors by distance
    keys = ['game_id', 'play_id', 'nfl_id']
    tmp['rnk'] = tmp.groupby(keys)['dist'].rank(method='first')
    if k_neigh is not None:
        tmp = tmp[tmp['rnk'] <= float(k_neigh)]
    
    # Distance weighting
    tmp['w'] = np.exp(-tmp['dist'] / float(tau))
    sum_w = tmp.groupby(keys)['w'].transform('sum')
    tmp['wn'] = np.where(sum_w > 0, tmp['w'] / sum_w, 0.0)
    
    # Separate ally/opponent weights
    tmp['wn_ally'] = tmp['wn'] * tmp['is_ally']
    tmp['wn_opp'] = tmp['wn'] * (1.0 - tmp['is_ally'])
    
    # Weighted relative features
    for col in ['dx', 'dy', 'dvx', 'dvy']:
        tmp[f'{col}_ally_w'] = tmp[col] * tmp['wn_ally']
        tmp[f'{col}_opp_w'] = tmp[col] * tmp['wn_opp']
    
    tmp['dist_ally'] = np.where(tmp['is_ally'] > 0.5, tmp['dist'], np.nan)
    tmp['dist_opp'] = np.where(tmp['is_ally'] < 0.5, tmp['dist'], np.nan)
    
    # Aggregate
    ag = tmp.groupby(keys).agg(
        gnn_ally_dx_mean=('dx_ally_w', 'sum'),
        gnn_ally_dy_mean=('dy_ally_w', 'sum'),
        gnn_ally_dvx_mean=('dvx_ally_w', 'sum'),
        gnn_ally_dvy_mean=('dvy_ally_w', 'sum'),
        gnn_opp_dx_mean=('dx_opp_w', 'sum'),
        gnn_opp_dy_mean=('dy_opp_w', 'sum'),
        gnn_opp_dvx_mean=('dvx_opp_w', 'sum'),
        gnn_opp_dvy_mean=('dvy_opp_w', 'sum'),
        gnn_ally_cnt=('is_ally', 'sum'),
        gnn_opp_cnt=('is_ally', lambda s: float(len(s) - s.sum())),
        gnn_ally_dmin=('dist_ally', 'min'),
        gnn_ally_dmean=('dist_ally', 'mean'),
        gnn_opp_dmin=('dist_opp', 'min'),
        gnn_opp_dmean=('dist_opp', 'mean'),
    ).reset_index()
    
    # Fill missing
    for c in ag.columns:
        if c not in keys:
            ag[c] = ag[c].fillna(0.0 if 'cnt' in c or 'mean' in c else radius)
    
    return ag

print('GNN embedding function defined')

GNN embedding function defined


## 4. Route Pattern Extraction

In [5]:
def extract_route_patterns(input_df, kmeans=None, scaler=None, fit=True):
    """Cluster trajectory patterns using KMeans"""
    route_features = []
    
    for (gid, pid, nid), group in tqdm(input_df.groupby(['game_id', 'play_id', 'nfl_id']), 
                                        desc='Extracting routes', leave=False):
        traj = group.sort_values('frame_id').tail(5)
        
        if len(traj) < 3:
            continue
        
        positions = traj[['x', 'y']].values
        speeds = traj['s'].values
        
        # Trajectory metrics
        total_dist = np.sum(np.sqrt(np.diff(positions[:, 0])**2 + np.diff(positions[:, 1])**2))
        displacement = np.sqrt((positions[-1, 0] - positions[0, 0])**2 + 
                              (positions[-1, 1] - positions[0, 1])**2)
        straightness = displacement / (total_dist + 0.1)
        
        # Turn analysis
        angles = np.arctan2(np.diff(positions[:, 1]), np.diff(positions[:, 0]))
        if len(angles) > 1:
            angle_changes = np.abs(np.diff(angles))
            max_turn = np.max(angle_changes)
            mean_turn = np.mean(angle_changes)
        else:
            max_turn = mean_turn = 0
        
        route_features.append({
            'game_id': gid, 'play_id': pid, 'nfl_id': nid,
            'traj_straightness': straightness,
            'traj_max_turn': max_turn,
            'traj_mean_turn': mean_turn,
            'traj_depth': abs(positions[-1, 0] - positions[0, 0]),
            'traj_width': abs(positions[-1, 1] - positions[0, 1]),
            'speed_mean': speeds.mean(),
            'speed_change': speeds[-1] - speeds[0] if len(speeds) > 1 else 0,
        })
    
    route_df = pd.DataFrame(route_features)
    feat_cols = ['traj_straightness', 'traj_max_turn', 'traj_mean_turn',
                 'traj_depth', 'traj_width', 'speed_mean', 'speed_change']
    X = route_df[feat_cols].fillna(0)
    
    if fit:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        kmeans = KMeans(n_clusters=cfg.N_ROUTE_CLUSTERS, random_state=cfg.SEED, n_init=10)
        route_df['route_pattern'] = kmeans.fit_predict(X_scaled)
        return route_df, kmeans, scaler
    else:
        X_scaled = scaler.transform(X)
        route_df['route_pattern'] = kmeans.predict(X_scaled)
        return route_df

print('Route pattern extraction defined')

Route pattern extraction defined


## 5. Model Architecture - GRU + Attention

In [6]:
class JointSeqModel(nn.Module):
    """Proven GRU + Attention architecture"""
    
    def __init__(self, input_dim, horizon):
        super().__init__()
        self.gru = nn.GRU(input_dim, 128, num_layers=2, batch_first=True, dropout=0.1)
        self.pool_ln = nn.LayerNorm(128)
        self.pool_attn = nn.MultiheadAttention(128, num_heads=4, batch_first=True)
        self.pool_query = nn.Parameter(torch.randn(1, 1, 128))
        
        self.head = nn.Sequential(
            nn.Linear(128, 256), 
            nn.GELU(), 
            nn.Dropout(0.2), 
            nn.Linear(256, horizon * 2)
        )
    
    def forward(self, x):
        h, _ = self.gru(x)
        B = h.size(0)
        q = self.pool_query.expand(B, -1, -1)
        ctx, _ = self.pool_attn(q, self.pool_ln(h), self.pool_ln(h))
        out = self.head(ctx.squeeze(1))
        out = out.view(B, -1, 2)
        return torch.cumsum(out, dim=1)

# Test model
model = JointSeqModel(167, cfg.MAX_FUTURE_HORIZON).to(cfg.DEVICE)
print(f'Model parameters: {sum(p.numel() for p in model.parameters()):,}')

Model parameters: 360,892


## 6. Loss Function - Temporal Huber

In [7]:
class TemporalHuber(nn.Module):
    """Huber loss with time decay weighting"""
    def __init__(self, delta=0.5, time_decay=0.03):
        super().__init__()
        self.delta = delta
        self.time_decay = time_decay
    
    def forward(self, pred, target, mask):
        err = pred - target
        abs_err = torch.abs(err)
        huber = torch.where(abs_err <= self.delta, 
                           0.5 * err * err, 
                           self.delta * (abs_err - 0.5 * self.delta))
        
        if self.time_decay > 0:
            L = pred.size(1)
            t = torch.arange(L, device=pred.device).float()
            weight = torch.exp(-self.time_decay * t).view(1, L, 1)
            huber = huber * weight
            mask = mask.unsqueeze(-1) * weight
        
        return (huber * mask).sum() / (mask.sum() + 1e-8)

criterion = TemporalHuber(delta=0.5, time_decay=0.03)
print('Loss function defined')

Loss function defined


## 7. Load Data

Load training data (using 2 weeks for demo, use all 18 for production).

In [8]:
# Load data (demo: 2 weeks)
weeks_to_load = [1, 2]  # Use range(1, 19) for full training

print('Loading training data...')
input_dfs, output_dfs = [], []

for week in weeks_to_load:
    input_file = cfg.DATA_DIR / f'input_2023_w{week:02d}.csv'
    output_file = cfg.DATA_DIR / f'output_2023_w{week:02d}.csv'
    
    if input_file.exists() and output_file.exists():
        input_dfs.append(pd.read_csv(input_file))
        output_dfs.append(pd.read_csv(output_file))
        print(f'  Week {week}')

train_input = pd.concat(input_dfs, ignore_index=True)
train_output = pd.concat(output_dfs, ignore_index=True)

print(f'\nLoaded:')
print(f'  Input:  {len(train_input):,} rows')
print(f'  Output: {len(train_output):,} rows')

Loading training data...


  Week 1


  Week 2

Loaded:
  Input:  574,300 rows
  Output: 64,268 rows


## 8. Summary

This notebook demonstrates the Geometric Neural Network approach:

1. **Geometric Baseline**: Compute deterministic endpoints based on player roles
2. **GNN Embeddings**: Distance-weighted neighbor feature aggregation
3. **Route Patterns**: K-means clustering of trajectory shapes
4. **167 Features**: 154 proven + 13 geometric breakthrough features
5. **GRU + Attention**: Proven architecture for sequence modeling

**Key Insight**: Train the model to learn CORRECTIONS to the geometric baseline, not raw predictions.

**Next Steps**:
- Run full feature engineering pipeline
- Train with 5-fold cross-validation
- See `06_gru_training.ipynb` for the full GRU implementation
- See `07_kaggle_submission.ipynb` for submission format